In [39]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

from scipy.ndimage.interpolation import zoom
from PIL import Image
from scipy.misc import imsave
import glob
from os.path import basename
from scipy.misc import imresize


from keras import Model
from keras.models import load_model
from keras.layers import Conv2D, Input, Dense,Flatten,LeakyReLU, BatchNormalization, MaxPooling2D, UpSampling2D,Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras import backend as K
import tensorflow as tf
import keras
print(K.backend())

from image_helper import *

tensorflow


In [40]:
augmented_train_data, augmented_train_label = load_augmented_data("./augmented_data/", "train")
augmented_validation_data, augmented_validation_label = load_augmented_data("./augmented_data/", "val")
augmented_test_data, augmented_test_label = load_augmented_data("./augmented_data/", "test")

partition_dict, labels_dict = get_generator_dicts("./augmented_data/")

In [41]:
resize_factor = 0.6
augmented_train_data_resized = np.zeros((augmented_train_data.shape[0], imresize(augmented_train_data[0], resize_factor).shape[0], imresize(augmented_train_data[0], resize_factor).shape[1], 3))
augmented_validation_data_resized = np.zeros((augmented_validation_data.shape[0], imresize(augmented_validation_data[0], resize_factor).shape[0], imresize(augmented_validation_data[0], resize_factor).shape[1], 3))


for i, im in enumerate(augmented_train_data):
    augmented_train_data_resized[i] = imresize(im, resize_factor)/255
    

for i, im in enumerate(augmented_validation_data):
    augmented_validation_data_resized[i] = imresize(im, resize_factor)/255
    #plot_with_label(augmented_validation_data_resized[i], augmented_validation_label[i][0], augmented_validation_label[i][1])
    
augmented_train_data = augmented_train_data_resized
augmented_validation_data = augmented_validation_data_resized

C:\Users\Luca\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
C:\Users\Luca\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Luca\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys
C:\Users\Luca\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instea

In [52]:
augmented_train_data_resized_target = np.zeros((augmented_train_data.shape[0], 200, 296, 3))
augmented_validation_data_resized_target = np.zeros((augmented_validation_data.shape[0], 200, 296, 3))


for i, im in enumerate(augmented_train_data):
    augmented_train_data_resized_target[i] = imresize(im, (200, 296))/255
    

for i, im in enumerate(augmented_validation_data):
    augmented_validation_data_resized_target[i] = imresize(im, (200, 296))/255

C:\Users\Luca\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
C:\Users\Luca\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # Remove the CWD from sys.path while we load stuff.


In [42]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=3,
                  shuffle=True, fully_convolutional = True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        self.path = path

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, 2), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = imresize(np.array(Image.open(self.path + ID + '.png')), resize_factor)/255
            

            # Store class
            y[i,0], y[i,1] = self.labels[ID]
            

        return X, y

In [53]:
tbCallBack = TensorBoard(log_dir='./logs/regression', histogram_freq=2, batch_size=32, write_graph=True, write_grads=True, write_images=True)
cpCallBack = ModelCheckpoint('./checkpoints/regression_auto/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=3)
esCallBack = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

K.clear_session()
sess = tf.Session()
K.set_session(sess)

In [54]:
input_img = Input(shape=(augmented_train_data[0].shape[0], augmented_train_data[0].shape[1], 3))

x = Conv2D(64, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid', name="aux_output")(x)

xx = Flatten()(encoded)
xx = Dense(8, activation='relu', use_bias=False)(xx)
output = Dense(2, use_bias=False, name="main_output")(xx)



model = Model(inputs=input_img, outputs=[output, decoded])

from keras import optimizers
#sgd = optimizers.SGD(lr=1e-11, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=1e-9)
model.compile(optimizer="adam", loss=['mean_squared_error', "binary_crossentropy"])

#tbCallBack.set_model(model)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 195, 294, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 195, 294, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 195, 294, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 195, 294, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [ ]:
params = {'dim': (augmented_train_data[0].shape[0],augmented_train_data[0].shape[1]),
          'batch_size': 32,
          'n_channels': 3,
          'shuffle': True}
training_generator = DataGenerator("./augmented_data/data/", partition_dict['train'], labels_dict, **params)
validation_generator = DataGenerator("./augmented_data/data/", partition_dict['val'], labels_dict, **params)

#history = model.fit_generator(generator=training_generator, validation_data=[augmented_validation_data, augmented_validation_label], epochs=3, callbacks=[cpCallBack, esCallBack])


history = model.fit(augmented_train_data, {'main_output': augmented_train_label, 'aux_output': augmented_train_data_resized_target}, validation_data=[augmented_validation_data, {'main_output': augmented_validation_label, 'aux_output': augmented_validation_data_resized_target}] ,epochs=30, batch_size=32, callbacks=[tbCallBack])

Train on 412 samples, validate on 80 samples


In [ ]:
predictions = model.predict(augmented_train_data_resized[:10])
print(predictions)
for i, pred in enumerate(predictions):
    plot_with_label(augmented_train_data_resized[i], pred[0], pred[1])

In [ ]:
predictions = model.predict(augmented_validation_data_resized)
for i, pred in enumerate(predictions):
    plot_with_label(augmented_validation_data_resized[i], pred[0], pred[1])